In [1]:
import os 
import pandas as pd 
from urllib.request import urlopen
import json

In [2]:
## read in api key
with open('../.fmp_api.key') as f:
    api_key = f.read().strip()

In [3]:
## parameters
start_date = '2015-01-01' 
end_date = '2024-12-31'
sp500 = '^GSPC'

In [4]:
## function to get data from api.
def get_fmp_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return pd.DataFrame(json.loads(data))


In [5]:
## get S&P 500 prices history.
url = f"https://financialmodelingprep.com/stable/historical-price-eod/full?symbol={sp500}&from={start_date}&to={end_date}&apikey={api_key}"
sppa = get_fmp_data(url) ## S&P 500 price action

In [83]:
## process S&P 500 data
sp500 = \
(
    sppa
    .loc[:, ['date', 'close']]
    .assign(date = lambda x: pd.to_datetime(x.date))
    .set_index('date')
    .sort_index(ascending=True)
    .assign(pctchg = lambda x: x.close.pct_change())
    .dropna()
    .resample('ME')
    .agg({'close': 'last',
          'pctchg': lambda x: (1 + x).prod() - 1})
    .reset_index()
    .assign(month = lambda df_: df_.date.dt.to_period('M'))
    .set_index('month')
)

In [17]:
## get S&P 500 constituents
url = f"https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={api_key}"
spc = get_fmp_data(url) ## current S&P 500 constituents

url = f"https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey={api_key}"
sphc = get_fmp_data(url) ## historical additions and removals

In [143]:
## start with the current constituents and work backwards to build monthly snapshots of the S&P 500 constituents.
sp500_constituents = {sp500.index[-1]: set(spc.symbol)}

sp500_changes = \
(
    sphc
    .loc[:, ['date', 'symbol', 'removedTicker']]
    .assign(month = lambda df_: pd.to_datetime(df_.date).dt.to_period('M'))
    .groupby('month')
    .agg({'symbol': set, 'removedTicker': set})
    .sort_index(ascending=False)
)

In [144]:
month_i = sp500.index[-1]
symbolsToRemove = sp500_changes.loc[month_i, 'symbol']
symbolsToAdd = sp500_changes.loc[month_i, 'removedTicker']
constituents_i = sp500_constituents[month_i].copy()

for month_i in sp500.index[-2:0:-1]:
    if month_i in sp500_changes.index:
        if symbolsToAdd:
            constituents_i.update(symbolsToAdd.copy())
        if symbolsToRemove:
            constituents_i.difference_update(symbolsToRemove.copy())
        constituents_i.discard('')

        symbolsToRemove = sp500_changes.loc[month_i, 'symbol']
        symbolsToAdd = sp500_changes.loc[month_i, 'removedTicker']

    sp500_constituents[month_i] = constituents_i.copy()

In [145]:
sp500_changes.loc[sp500.index[-8]]

symbol           {PXD, VST}
removedTicker       {, PXD}
Name: 2024-05, dtype: object

In [ ]:
sphc.iloc[:20]

,dateAdded,addedSecurity,removedTicker,removedSecurity,date,symbol,reason
0,"December 23, 2024","Workday, Inc.",AMTM,Amentum,2024-12-22,WDAY,Market capitalization change.
1,"December 23, 2024",Lennox International,CTLT,Catalent,2024-12-22,LII,Acquired by Novo Holdings A/S (NVO)
2,"December 23, 2024",Apollo Global Management,QRVO,Qorvo,2024-12-22,APO,Market capitalization change.
3,"November 26, 2024",Texas Pacific Land Corporation,MRO,Marathon Oil,2024-11-25,TPL,ConocoPhillips acquired Marathon Oil.\n
4,"October 30, 2024",Amentum,BBWI,"Bath & Body Works, Inc.",2024-10-30,AMTM,Market capitalization change.
5,"September 23, 2024",Erie Indemnity,BIO,Bio-Rad Laboratories,2024-09-23,ERIE,Market capitalization change.
6,"September 23, 2024",Dell Technologies,ETSY,Etsy,2024-09-23,DELL,Market capitalization change.
7,"September 23, 2024",Palantir Technologies,AAL,American Airlines Group,2024-09-23,PLTR,Market capitalization change.
8,"July 05, 2024",Smurfit WestRock PLC,WRK,WestRock Co.,2024-07-05,SW,Smurfit Kappa and WestRock combined under SW.
9,"June 24, 2024",,ILMN,"Illumina, Inc.",2024-06-24,ILMN,Market capitalization change.


In [133]:

i=50
sp500_constituents[sp500.index[i]], len(sp500_constituents[sp500.index[i]])

({'A',
  'AAL',
  'AAP',
  'AAPL',
  'ABBV',
  'ABMD',
  'ABT',
  'ACN',
  'ADBE',
  'ADI',
  'ADM',
  'ADP',
  'ADS',
  'ADSK',
  'AEE',
  'AEP',
  'AES',
  'AFL',
  'AGN',
  'AIG',
  'AIV',
  'AIZ',
  'AJG',
  'AKAM',
  'ALB',
  'ALGN',
  'ALK',
  'ALL',
  'ALLE',
  'ALXN',
  'AMAT',
  'AMD',
  'AME',
  'AMG',
  'AMGN',
  'AMP',
  'AMT',
  'AMZN',
  'ANET',
  'ANSS',
  'AON',
  'AOS',
  'APA',
  'APC',
  'APD',
  'APH',
  'APTV',
  'ARE',
  'ATO',
  'ATVI',
  'AVB',
  'AVGO',
  'AVY',
  'AWK',
  'AXP',
  'AZO',
  'BA',
  'BAC',
  'BALL',
  'BAX',
  'BBWI',
  'BBY',
  'BDX',
  'BEN',
  'BF-B',
  'BIIB',
  'BK',
  'BKNG',
  'BKR',
  'BLK',
  'BMY',
  'BR',
  'BRK-B',
  'BSX',
  'BWA',
  'BXP',
  'C',
  'CAG',
  'CAH',
  'CAT',
  'CB',
  'CBOE',
  'CBRE',
  'CCI',
  'CCL',
  'CDNS',
  'CE',
  'CELG',
  'CERN',
  'CF',
  'CFG',
  'CHD',
  'CHRW',
  'CHTR',
  'CI',
  'CINF',
  'CL',
  'CLX',
  'CMCSA',
  'CME',
  'CMG',
  'CMI',
  'CMS',
  'CNC',
  'CNP',
  'COF',
  'COO',
  'COP',
  'COR

In [ ]:
sp500_changes.iloc[50:70]

,symbol,removedTicker
month,,
2019-04,"{DOW, BHF}","{, BHF}"
2019-02,"{ATO, WAB}","{GT, NFX}"
2019-01,{TFX},{PCG}
2018-12,"{FRC, CE}","{SCG, ESRX}"
2018-11,"{KEYS, JKHY, FANG, LW, MXIM}","{EQT, SRCL, CA, COL, AET}"
2018-10,{FTNT},{EVHC}
2018-09,"{ROL, WCG}","{XL, ANDV}"
2018-08,{ANET},{GGP}
2018-06,"{HFC, TWTR, CPAY, EVRG, CPRT, BR}","{AYI, TWX, DPS, MON, RRC, NAVI}"


In [146]:
[k for k,v in sp500_constituents.items() if 'DOW' in v]

[Period('2024-12', 'M'),
 Period('2024-11', 'M'),
 Period('2024-10', 'M'),
 Period('2024-09', 'M'),
 Period('2024-08', 'M'),
 Period('2024-07', 'M'),
 Period('2024-06', 'M'),
 Period('2024-05', 'M'),
 Period('2024-04', 'M'),
 Period('2024-03', 'M'),
 Period('2024-02', 'M'),
 Period('2024-01', 'M'),
 Period('2023-12', 'M'),
 Period('2023-11', 'M'),
 Period('2023-10', 'M'),
 Period('2023-09', 'M'),
 Period('2023-08', 'M'),
 Period('2023-07', 'M'),
 Period('2023-06', 'M'),
 Period('2023-05', 'M'),
 Period('2023-04', 'M'),
 Period('2023-03', 'M'),
 Period('2023-02', 'M'),
 Period('2023-01', 'M'),
 Period('2022-12', 'M'),
 Period('2022-11', 'M'),
 Period('2022-10', 'M'),
 Period('2022-09', 'M'),
 Period('2022-08', 'M'),
 Period('2022-07', 'M'),
 Period('2022-06', 'M'),
 Period('2022-05', 'M'),
 Period('2022-04', 'M'),
 Period('2022-03', 'M'),
 Period('2022-02', 'M'),
 Period('2022-01', 'M'),
 Period('2021-12', 'M'),
 Period('2021-11', 'M'),
 Period('2021-10', 'M'),
 Period('2021-09', 'M'),
